### IS - Brand position
Finding Product position on Semantic Network (PPSN)

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import itertools
import math
import string
import json
import pymongo
from itertools import groupby

In [2]:
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=pd.core.common.SettingWithCopyWarning)

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\ProgramData\Anaconda3\lib\site-packages\_pytest\mark\structures.py:378: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)


In [3]:
import time
from datetime import datetime
today = datetime.now()

In [4]:
from pythainlp.util import normalize
from newnewthaicut import word_tokenize  # IOB Tagging tokenized
from pythainlp.corpus import thai_stopwords
from pythainlp.spell import correct

In [5]:
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import jaccard_score
import scipy.stats
from scipy.spatial.distance import cosine
from mlxtend.evaluate import lift_score
from gensim.models import TfidfModel

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
# Files definition
root_input_path = 'Datasource'
root_output_path = 'output'

#comment_file = []
#comment_file.append(root_input_path+"/comment_meiji.csv")
#comment_file.append(root_input_path+"/comment_DDCF.csv")

#comment_file = []
#comment_file.append("comment_meiji.csv")
#comment_file.append("comment_DDCF.csv")

comment_nlp_file = root_output_path+"/comment_nlpToken.csv"
comment_cooc_freq_file = root_output_path+"/comment_cooc_freq.xlsx"
comment_cooc_jacc_file = root_output_path+"/comment_cooc_jacc.xlsx"

comment_robust_jaccard = root_output_path+"/comment_robust_jaccard.xlsx"
comment_robust_lift = root_output_path+"/comment_robust_lift.xlsx"
comment_robust_incur = root_output_path+"/comment_robust_incur.xlsx"
comment_robust_cosine = root_output_path+"/comment_robust_cosine.xlsx"

#comment_nlp_file = "comment_nlpToken.csv"
#comment_cooc_nlp_file = "comment_cooc.xlsx"

In [7]:
# Connect MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["NIDA_PPSN_PRD"]
col_thread = mydb["NIDA_PPSN_THREAD"]
col_comment = mydb["NIDA_PPSN_COMMENT"]
col_scrape = mydb["NIDA_PPSN_SCRAPE"]

### 1.Clean data & NLP Tokenized

In [ ]:
cursor_corpus = col_scrape.find()
df_Corpus = pd.DataFrame(cursor_corpus)

In [ ]:
df_Corpus.head()

In [ ]:
print('All comment:',df_Corpus.shape[0])
print('All URLs ก่อนตัด:',len(df_Corpus.URLs.unique()))

In [ ]:
missing = df_Corpus.isnull().sum()
print(missing)

In [ ]:
df_Corpus.tail()

In [ ]:
df_Corpus.drop('_id',axis=1,inplace=True)

In [ ]:
seqNum = list(range(1,df_Corpus.shape[0]+1))
df_Corpus.insert(0,'commentId',seqNum)
df_Corpus.columns = ['commentId','URLs','headline','text','article_date','Retrived_date']

In [ ]:
df_Corpus.tail()

In [ ]:
# Drop NA & banned comments
df_Corpus = df_Corpus.dropna()
df_Corpus = df_Corpus[df_Corpus["text"].str.find('ความคิดเห็นนี้ถูกลบ')!= 0]
df_Corpus = df_Corpus[df_Corpus["text"].str.find('แก้ไขข้อความเมื่อ')!= 0]
df_Corpus = df_Corpus.set_index('commentId')

In [ ]:
missing = df_Corpus.isnull().sum()
print(missing)

In [ ]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

In [ ]:
df_Corpus.insert(4,'token_text',None)
df_Corpus.tail()

เช็ควันเวลา

In [ ]:
print('กระทู้เก่าที่สุด :',min(df_Corpus.article_date))
print('กระทู้ใหม่ที่สุด :',max(df_Corpus.article_date))

เก็บเฉพาะหัวกระทู้

In [ ]:
df_thread = df_Corpus.copy()
df_thread = df_thread[['URLs','headline']]
df_thread.drop_duplicates(subset ="URLs", keep = 'first', inplace = True)
df_thread.insert(2,'token_headline',None)
df_thread.insert(3,'mention_product',None)
df_thread.insert(4,'mention_brand',None)
df_thread.tail()

## 2.ตัดคำและเก็บลง MongoDB (ใช้ insight จาก jupyter งานก่อน)

https://www.thainlp.org/pythainlp/tutorials/notebooks/pythainlp-get-started.html#Thai-Characters

In [ ]:
def normThai(w):
    returnList = []
    for i in w:
        returnList.append(normalize(i))
    return returnList

In [ ]:
garbage_char = ['ไม่','','.','..','...','มกราคม', 'กุมภาพันธ์','มีนาคม', 'เมษายน','พฤษภาคม', 'มิถุนายน', 'กรกฎาคม','สิงหาคม','กันยายน'
                ,'ตุลาคม','พฤศจิกายน', 'ธันวาคม','วันจันทร์','วันอังคาร','วันพุธ','วันพฤหัสบดี','วันศุกร์','วันเสาร์','วันอาทิตย์','กก'
                ,'เมนู','Menu','Net','net','สาขา','บาท','ราคา','ฯ','ๆ','กก','อันนี้','😆', '🤣','😢','😏','😂','😿','🥺','ววว','xx','อิอิ','แย้ววว']
naka = ['นะคะ','นะค่ะ','น่ะค่ะ','น่ะคะ','ฮ้าฟ','ค้าบ','คร้าบ']

stopwords = set(thai_stopwords()).union(set(naka))
stopwords.remove('ไม่')
stopwords.remove('สูง')

In [ ]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

In [ ]:
def tokenize_custom(x):
    custom_punc = '!.—-"$#&฿/&\'()*+,:;<=>?@[\\]^_`{|}~'
    x = x.translate(str.maketrans('', '', custom_punc)).strip()
    x = x.translate(str.maketrans({"\t":None,"\n": None})).strip()
    x = x.lower()
    wtkn = word_tokenize(x)
    wtkn = [j for j in wtkn if j not in stopwords]
    wtkn = [w for w in wtkn if len(w) >= 2]
    wtkn = [y for y in wtkn if y not in garbage_char]
    wtkn = [s for s in wtkn if not s.isdigit()]
    wtkn = normThai(wtkn)
    return wtkn

In [ ]:
def create_InsertMany_Thread(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "URLs": lx,
            "headline": df.headline[c],
            "token_headline":df.token_headline[c],
            "t_mention_dairy":df.t_mention_dairy[c],
            "t_mention_product": df.t_mention_product[c],
            "t_mention_brand": df.t_mention_brand[c]
        }
        listofdict.append(info)
    return listofdict

In [ ]:
def create_InsertMany_comments(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "commentId":int(df.commentId[c]),
            "URLs": lx,
            "headline":df.headline[c],
            "text": df.text[c],
            "Retrived_date": df.Retrived_date[c],
            "token_text":df.token_text[c],
            "cmt_mention_dairy": df.cmt_mention_dairy[c],
            "cmt_mention_product": df.cmt_mention_product[c],
            "cmt_mention_brand": df.cmt_mention_brand[c]
        }
        listofdict.append(info)
    return listofdict

In [ ]:
def generalize_keyword(wtkn):  
    ret_wtkn = wtkn
    for i,each in enumerate(ret_wtkn,0):
        if each in ['ดัชมิลล์','ดัชมิล','ดัชมิว','dutchmill','dutch mill','dutch milk','dutchmilk','duchmill','duchie-bio','ดัชชี่']:
            ret_wtkn[i] = 'ดัชมิลล์'
        elif each  in ['เมจิ','ซีพีเมจิ','ซีพี เมจิ','cp meiji','meiji','meji','miji','cpmeijicom','cpmeijitensai','cpmeijithailand']:
            ret_wtkn[i] = 'เมจิ'
        elif each  in ['โฟร์โมสต์','โฟร์โมส','โฟโมสต์','โฟโมต','โฟรโมสต์','โฟรโมสต','foremost']:
            ret_wtkn[i] = 'โฟร์โมสต์'
        elif each  in ['แดรี่โฮม','dairy home','dairyhome','เดรี่โฮม','เดลี่โฮม']:
            ret_wtkn[i] = 'แดรี่โฮม'
        elif each  in ['โชคชัย','อืมม มิลค์','umm milk','นมฟาร์มโชคชัย']:
            ret_wtkn[i] = 'โชคชัย'
        elif each  in ['เอ็มมิลค์','mmilk','เอ็มมิ้ลค์','เอ็มมิลล์','m milk']:
            ret_wtkn[i] = 'เอ็มมิลค์'
        elif each  in ['ไทยเดนมาร์ค','ไทยเดนมาร์ก','ไทย เดนมาร์ค','ไทย เดนมาร์ก','วัวแดง']:
            ret_wtkn[i] = 'ไทยเดนมาร์ค'
        elif each  in ['สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่','สตรอเบอร์รี่']:
            ret_wtkn[i] = 'สตรอว์เบอร์รี'
        elif each  in ['ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','ช็อค']:
            ret_wtkn[i] = 'ช็อกโกแลต'
        elif each  in ['มิ้นท์ชอค','มิ้นต์ช้อก','ช็อคโกแลตมินต์','มินต์ช็อคโกแลต']:
            ret_wtkn[i] = 'ช็อกโกแลตมินต์'
        elif each  in ['ไขมันต่ำ','low fat','พร่องมันเนย']:
            ret_wtkn[i] = 'ไขมันต่ำ'
        elif each  in ['ไขมัน 0','ไขมัน 0%','0 fat','0% fat','ขาดมันเนย']:
            ret_wtkn[i] = 'ไขมัน 0%'
        elif each  in ['high protein','hi protein','ไฮโปรตีน']:
            ret_wtkn[i] = 'ไฮโปรตีน'
        elif each  in ['free lactose','lactose free','ฟรีแลคโตส','แลคโตสฟรี']:
            ret_wtkn[i] = 'นมฟรีแลคโตส'
        elif each  in ['พาส','พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์']:
            ret_wtkn[i] = 'พาสเจอร์ไรส์'
        elif each  in ['เมจิโกลด์ แม็กซ์','เมจิโกลด์','เมจิ โกลด์','เมจิโกล์ดแม็กซ์','gold','gold max']:
            ret_wtkn[i] = 'เมจิโกลด์'
        elif each  in ['ฝาน้ำเงิน','รสจืด']:
            ret_wtkn[i] = 'รสจืด'
        elif each  in ['ท็อปส์','ทอปส์','ท้อปส์','ท๊อปส์','tops']:
            ret_wtkn[i] = 'tops'
        elif each  in ['แมคโคร','แม็คโคร','makro']:
            ret_wtkn[i] = 'makro'
        elif each  in ['โลตัส','lotus']:
            ret_wtkn[i] = 'lotus'
        elif each  in ['บิ๊กซี','bigc','big c']:
            ret_wtkn[i] = 'bigc'
        elif each in ['ดาร์คช็อกโกแลต','ดาร์กช็อก','ดาร์กช็อกโกแลต']:
            ret_wtkn[i] = 'ดาร์คช็อกโกแลต'
        elif each in ['bedtime','bed time','เบดไทม์']:
            ret_wtkn[i] = 'เบดไทม์'
        elif each in ['เมจิ บัลแกเรีย','เมจิบัลแกเรีย','บัลแกเรีย','bulgaria']:
            ret_wtkn[i] = 'เมจิบัลแกเรีย'
        elif each in ['โยเกิร์ต','โยเกิต','โยเกิรต','โยเกิร์ตพร้อมดื่ม']:
            ret_wtkn[i] = 'โยเกิร์ต'
        elif each in ['7 Eleven','7 eleven','เซเว่น อีเลฟเว่น','เซเว่นอีเลฟเว่น','เซเว่น','เซเวน','7 11']:
            ret_wtkn[i] = '7-Eleven'
        elif each in ['บีทาเก้น','บีทาเกน']:
            ret_wtkn[i] = 'บีทาเก้น'
        elif each in ['โปร','โปรโมชั่น']:
            ret_wtkn[i] = 'โปรโมชั่น'
    return ret_wtkn

In [ ]:
def find_brands_product(x,catType):
    ret_wtkn = []
    brands = ['ดัชมิลล์','เมจิ','โฟร์โมสต์','โชคชัย','แดรี่โฮม','เอ็มมิลค์','แมคโนเลีย','ไทยเดนมาร์ค','หนองโพ','คาเนชั่น','บีทาเก้น','จิตรลดา']
    products = ['สตรอว์เบอร์รี','ช็อกโกแลต','รสกาแฟ','รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','ไฮโปรตีน','อัลมอนด์'
                ,'รสกล้วย','grass fed','นมฟรีแลคโตส','เมจิโกลด์','นมฮอกไกโด','เบดไทม์','ดาร์คช็อกโกแลต','ไฮแคลเซียม'
                ,'คาราเมล','มอลต์','เมล่อน','ชาเขียวมัจฉะ','บัลแกเรีย','รสธรรมชาติ','รสกลมกล่อม','ซากุระ','วิปครีม']
    milk_kind=['นม','นมข้น','นมจืด','นมสด','กินนม','ดื่มนม','ขวดนม','นมวัว','นมกล่อง','ผลิตภันท์นม','น้ำนมโค'
                     ,'โยเกิร์ต','นมเปรี้ยว','uht','นมถั่วเหลือง','นมผง','พาสเจอร์ไรส์','nondairy','non dairy']
    
    choiceList = []
    if catType == 'b':
        choiceList = brands
    elif catType == 'm':
        choiceList = milk_kind
    elif catType == 'p':
        choiceList = products
        
    for i,each in enumerate(x,0):
        if each in choiceList:
            ret_wtkn.append(each)
    return ret_wtkn

#### Copy ชุดคำจาก File มา Process ก่อนลง MongoDB

In [ ]:
df_process_comment = df_Corpus.reset_index().copy()
df_process_thread = df_thread.reset_index().copy()

2.1) Tokenize comment

In [ ]:
df_process_comment['token_text'] = df_process_comment['text'].apply(lambda x: tokenize_custom(x))

In [ ]:
df_process_comment.head()

2.2) Generalized keyword (Brand & product)

In [ ]:
df_process_comment['token_text'] = df_process_comment['token_text'].apply(lambda x: generalize_keyword(x))

In [ ]:
df_process_comment['cmt_mention_dairy'] = df_process_comment['token_text'].apply(lambda x: find_brands_product(x,'m'))
df_process_comment['cmt_mention_brand'] = df_process_comment['token_text'].apply(lambda x: find_brands_product(x,'b'))
df_process_comment['cmt_mention_product'] = df_process_comment['token_text'].apply(lambda x: find_brands_product(x,'p'))

2.3) Insert comments to DB

In [ ]:
col_comment.delete_many({})
process_data = create_InsertMany_comments(df_process_comment)
col_comment.insert_many(process_data)

2.4) Tokenize head of thread

In [ ]:
df_process_thread['token_headline'] = df_process_thread['headline'].apply(lambda x: tokenize_custom(x))

2.5) Generalized keyword (Brand & product)

In [ ]:
df_process_thread['token_headline'] = df_process_thread['token_headline'].apply(lambda x: generalize_keyword(x))

2.6) Mark domain/entity mention

In [ ]:
df_process_thread['t_mention_dairy'] = df_process_thread['token_headline'].apply(lambda x: find_brands_product(x,'m'))
df_process_thread['t_mention_brand'] = df_process_thread['token_headline'].apply(lambda x: find_brands_product(x,'b'))
df_process_thread['t_mention_product'] = df_process_thread['token_headline'].apply(lambda x: find_brands_product(x,'p'))

In [ ]:
df_process_thread.head()

2.7) Insert thread to DB

In [ ]:
col_thread.delete_many({})
process_data = create_InsertMany_Thread(df_process_thread)
col_thread.insert_many(process_data)

In [ ]:
del df_thread,df_Corpus,df_process_comment,df_process_thread

### 3.Process หัวกระทู้ เพื่อทำ Topic Modeling

เคสหลุดเงื่อนไข แต่ต้องใช้
1. https://pantip.com/topic/30833944
2. https://pantip.com/topic/30105850
4. https://pantip.com/topic/35439062

In [ ]:
execption_urls = ['https://pantip.com/topic/30833944','https://pantip.com/topic/30105850','https://pantip.com/topic/35439062']

In [ ]:
cursor_thread = col_thread.find()
df_thr_process = pd.DataFrame(cursor_thread)

In [ ]:
cursor_comment = col_comment.find()
df_cmt_process = pd.DataFrame(cursor_comment)

In [ ]:
df_urls_milk = df_thr_process[(df_thr_process.t_mention_dairy.str.len() != 0) | (df_thr_process.URLs.isin(execption_urls))]
df_urls_milk['t_mention_dairy'] = df_urls_milk['t_mention_dairy'].apply(lambda x: repr(set(x)))
df_urls_milk['t_mention_brand'] = df_urls_milk['t_mention_brand'].apply(lambda x: repr(set(x)))
df_urls_milk['t_mention_product'] = df_urls_milk['t_mention_product'].apply(lambda x: repr(set(x)))

In [ ]:
df_c_urls_milk = df_cmt_process[(df_cmt_process.cmt_mention_dairy.str.len() != 0) | (df_cmt_process.URLs.isin(execption_urls))]
df_c_urls_milk['cmt_mention_dairy'] = df_c_urls_milk['cmt_mention_dairy'].apply(lambda x: repr(set(x)))
df_c_urls_milk['cmt_mention_brand'] = df_c_urls_milk['cmt_mention_brand'].apply(lambda x: repr(set(x)))
df_c_urls_milk['cmt_mention_product'] = df_c_urls_milk['cmt_mention_product'].apply(lambda x: repr(set(x)))

In [ ]:
print('จำนวนกระทู้ที่มีการพูดถึงนม:',df_urls_milk.shape[0])
print('%จำนวนกระทู้ที่มีการพูดถึงนมเทียบทั้งหมด:',round((df_urls_milk.shape[0]/df_thr_process.shape[0])*100,2))
print('จำนวน comment ที่มีการพูดถึงนม:',df_c_urls_milk.shape[0])
print('%จำนวน comment ที่มีการพูดถึงนมเทียบทั้งหมด:',round((df_c_urls_milk.shape[0]/df_cmt_process.shape[0])*100,2))

* จำนวน comment ที่มีการพูดถึงนม ไม่ได้ impute มาจากหัวกระทู้ ถ้า impute จะมีเยอะกว่านี้อีก

### EDA#2

การระบุ domain ของนม บนหัวกระทู้

การระบุชื่อ Product บนหัวกระทู้ (Headline)

การระบุชื่อ Product ใน comment ทั้งใต้หัวกระทู้ ที่มี domain ของนม และส่วน comment<br>
อย่างไรก็ตามยังไม่ได้ impute domain จากหัวกระทู้เข้าไป

การระบุชื่อ brand ใน comment

### 4.สรุป Term & reduced_keyword

In [ ]:
execption_urls = ['https://pantip.com/topic/30833944','https://pantip.com/topic/30105850','https://pantip.com/topic/35439062']

In [ ]:
attrributes = ['เก็บรักษา','เข้มข้น','เค้ก','เจมส์','เจมส์ จิ','เจมส์จิ','เจลลาติน','เจือจาง','เชื้อ','เชื้อโรค','เชื้อจุลินทรีย์','เชื้อรา','เด็ก','เด็กเล็ก','เด็กแรกเกิด'
               ,'เด็กโต','เด็กชาย','เด็กทารก','เด็กน้อย','เด็กวัยรุ่น','เด็กอ่อน','เติบโต','เตี้ย','เนย','เนยแข็ง','เนยสด','เนสเล่','เนสกาแฟ','เนสท์เล่','เน่า','เน่าเสีย'
               ,'เบาหวาน','เมจิ','เมจิโกลด์','เมจิบัลแกเรีย','เมล่อน','เย็น','เล่นเวท','เล่นกล้าม','เลี้ยง','เลี้ยงเด็ก','เลี้ยงดู','เลี้ยงลูก','เลี้ยงสัตว์','เวย์','เวย์โปรตีน'
               ,'เหม็นเขียว','เหม็นเปรี้ยว','เหม็นคาว','เหม็นบูด','เอนไซม์','เอ็นไซม์','เอนฟาโกร','เอ็มมิลค์','เอสเปรสโซ','เอสเปรสโซ่','เอสเพรสโซ','เอสเพรสโซ่'
               ,'แข็งแรง','แคนตาลูป','แคลเซียม','แคลเซี่ยม','แคลอรี','แคลอรี่','แจก','แช่เย็น','แชร์','แดรี่โฮม','แตงโม','แถม','แบคทีเรีย','แบบสอบถาม','แพ็ก'
               ,'แพกเกจ','แพ็กเกจ','แพ็ค','แพ๊ค','แพคเกจ','แพ็คเกจ','แพคคู่','แพ้ท้อง','แพ้นม','แพ้นมวัว','แม่','แมกนีเซียม','แมคโนเลีย','แม่ลูกอ่อน','แร่ธาตุ'
               ,'แลกโตส','แล็กโต๊ส','แลคโตส','แลคติก','แอโรบิค','แอนตี้ไบโอติก','แอปเปิล','แอปเปิ้ล','โกโก้','โฆษณา','โชคชัย','โซเดียม','โด้บ','โด๊บ','โด้ป'
               ,'โด๊ป','โด้ปนม','โปรโมชั่น','โปรตีน','โพแทสเซียม','โฟเลต','โฟร์โมสต์','โฟลิค','โภชนาการ','โยเกิร์ต','โยเกิร์ท','โรคเบาหวาน','โรคกระเพาะ'
               ,'โรคขาดอาหาร','โรคท้องร่วง','โรคประจำตัว','โรคภัยไข้เจ็บ','โรคภูมิแพ้','โรคมะเร็ง','โรคหัวใจ','โรงเรียน','โรงงาน','โอวัลติน','ให้นม','ให้อาหาร'
               ,'ไขมัน','ไขมัน 0%','ไขมันต่ำ','ไทยเดนมาร์ค','ไอติม','ไอศกรีม','ไอศครีม','ไฮโปรตีน','กรดไขมัน','กรดอะมิโน','กระดูกพรุน','กระบวนการผลิต'
               ,'กระป๋อง','กล้วย','กล้วหอม','กล่อง','กลิ่น','กลิ่นคาว','กลิ่นหอม','กลูโคส','กาเฟอีน','กาแฟ','กาแฟเย็น','กาแฟร้อน','กาแฟสด','การบรรจุ'
               ,'การผลิต','การย่อยอาหาร','การลดน้ำหนัก','การลดราคา','การออกกำลังกาย','ข้น','ขนม','ขนมเค้ก','ขนมปัง','ขนมปังแผ่น','ขนมหวาน','ขนส่ง'
               ,'ขวดโหล','ขวดนม','ขวบ','ขาดตลาด','ขายไม่ดี','ขายไม่ออก','ขายปลีก','ขายส่ง','ข้าว','ข้าวโพด','ข้าวโอ๊ต','ข้าวกล้อง','คนรุ่นใหม่','คนวัยหนุ่มสาว'
               ,'คนสูงอายุ','ครรภ์','คลอเรสเตอรอล','คลอเลสเตอรอล','คลอโรฟิล','คลอดลูก','ความคุ้มค่า','ความมัน','ความร้อน','คอเลสเตอรอล','คาเฟอีน','ค่าขนส่ง'
               ,'คาปูชิโน','คาปูชิน่','คาร์โบไฮเดรต','คาราเมล','คาว','คุณแม่','คุณแม่มือใหม่','คุณค่า','คุณประโยชน์','คุณภาพ','คุณภาพดี','ฆ่าเชื้อ','จิตรลดา','จืด'
               ,'จุลินทรีย์','ฉลาก','ชง','ชงนม','ช็อกโกแลต','ช่องแช่แข็ง','ชอบ','ชา','ชาเขียว','ชาเขียวมัจฉะ','ชาเย็น','ชิม','ซื้อ','ซุปเปอร์มาร์เกต','ซุปเปอร์มาร์เก็ต'
               ,'ดัมิลล์','ดาร์คช็อกโกแลต','ต้ม','ตลาด','ตัวแทนจำหน่าย','ตู้เย็น','ท้องเสีย','ท้องตลาด','ท้องร่วง','ท้องว่าง','ท้องอืด','ทุเรียน','ธรรมชาติ','ธัญญาหาร'
               ,'ธัญพืช','ธาตุเหล็ก','นม','นมเปรี้ยว','นมกล่อง','นมข้น','นมข้นหวาน','นมควาย','นมถั่วเหลือง','นมผง','นมฟรีแลคโตส','นมยูเอชที','นมวัว','นมสด'
               ,'นมฮอกไกโด','นอน','น้ำ','นำเข้า','น้ำเชื่อม','น้ำเต้าหู้','น้ำตาล','น้ำตาลเทียม','น้ำตาลทราย','น้ำตาลทรายแดง','น้ำนม','น้ำผึ้ง','น้ำหนัก','บรรจุขวด'
               ,'บลูเบอร์รี่','บัลแกเรีย','บัลกาเรีย','บาริสตา','บาริสต้า','บีทาเก้น','บูด','ปริมาณ','ปลอดเชื้อ','ปวดท้อง','ปั่น','ผลไม้','ผสม','ผู้จัดจำหน่าย','ผู้ผลิต','ฝา'
               ,'พกพา','พนักงานขายนม','พลังงาน','พาราไดซ์','พารากอน','พาสเจอร์ไรส์','ฟรอยด์','ฟอง','ฟาร์ม','มอลต์','ยอดขาย','รสกลมกล่อม','รกล้วย','รสกาแฟ'
               ,'รสจืด','รสชาติ','รสหวาน','ราคาแพง','ราคาสูง','ร่างกาย','ร้านกาแฟ','รีวิว','ลดความอ้วน','ลดน้ำหนัก','ลดราคา','ลูก','ลูกชาย','ลูกสาว','วันหมดอายุ'
               ,'วานิลลา','วานิลล่า','วานิลา','วิ่ง','วิตามิน','สตรอเบอร์รี่','สตรอเบอรี','สตรอว์เบอร์รี','สต็อก','สต๊อก','สต็อค','สตาร์บัค','สยามพารากอน','สยามสแควร์'
               ,'สละ','ส่วนผสม','สะอาด','สัปปะรด','สารอาหาร','สำนักงานคณะกรรมการอาหารและยา','สิ่งเจือปน','สูง','สูงวัย','สูตร','หญ้า','หนองโพ','หมอ','หลอด'
               ,'หวาน','หอม','หางจระเข้','ห้างร้าน','อ้วน','ออกกำลังกาย','ออร์แกนิก','อะเมซอน','อาหาร','อาหารเสริม','อุณหภูมิ']

stores = ['tops','makro','lotus','bigc','7-Eleven']

products = ['สตรอว์เบอร์รี','ช็อกโกแลต','รสกาแฟ','รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','ไฮโปรตีน','อัลมอนด์'
                ,'รสกล้วย','grass fed','นมฟรีแลคโตส','เมจิโกลด์','นมฮอกไกโด','เบดไทม์','ดาร์คช็อกโกแลต','ไฮแคลเซียม'
                ,'คาราเมล','มอลต์','เมล่อน','ชาเขียวมัจฉะ','บัลแกเรีย','รสธรรมชาติ','รสกลมกล่อม','ซากุระ']
milk_kind=['นม','นมข้น','นมจืด','นมสด','กินนม','ดื่มนม','ขวดนม','นมวัว','นมกล่อง','ผลิตภันท์นม','น้ำนมโค'
                     ,'โยเกิร์ต','นมเปรี้ยว','uht','นมถั่วเหลือง','นมผง','พาสเจอร์ไรส์']

#  EDA ใน Excel ได้ flavor 68 ตัว (ถ้าดึงยี่ห้ออื่นก็เพิ่มอีก)
avai_flavs = ['เมจิเมจิโกลด์','เมจิเมล่อน','เมจิไขมัน 0%','เมจิไขมันต่ำ','เมจิไฮโปรตีน','เมจิช็อกโกแลต','เมจิชาเขียวมัจฉะ','เมจิดาร์คช็อกโกแลต'
 ,'เมจินมฟรีแลคโตส','เมจิบัลแกเรีย','เมจิมอลต์','เมจิรสกลมกล่อม','เมจิรสกล้วย','เมจิรกาแฟ','เมจิรสจืด','เมจิรสธรรมชาติ','เมจิรสหวาน'
 ,'เมจิสตรอว์เบอร์รี','เมจิอัลมอนด์','เอ็มมิลค์นมฟรีแลคโตส','เอ็มมิลค์รสจืด','แดรี่โฮมgrass fed','แดรี่โฮมเบดไทม์','แดรี่โฮมช็อกโกแลต'
 ,'แดรี่โฮมรสกล้วย','แดโฮมรสจืด','แดรี่โฮมรสหวาน','แดรี่โฮมสตรอว์เบอร์รี','แมคโนเลียไขมันต่ำ','แมคโนเลียช็อกโกแลต','แมคโนเลียรสจืด'
 ,'โชคชัยไขมันต่ำ','โชคชัยช็อกโกแลต','โชคชัยรสกาแฟ','โชคชัยรสจืด','โชคชัยสตรอว์เบอร์รี','ฟร์โมสต์ไขมัน 0%','โฟร์โมสต์ไขมันต่ำ'
 ,'โฟร์โมสต์คาราเมล','โฟร์โมสต์ช็อกโกแลต','โฟร์โมสต์รสกาแฟ','โฟร์โมสต์รสจืด','โฟร์โมสต์สตรอว์เบอร์รี','ไทยเดนมาร์คช็อกโกแลต'
 ,'ไทยเดนมาร์ครสกาแฟ','ไทยเดนมาร์ครสจืด','ไทยเดนมาร์ครสหวาน','ไทยเดนมาร์คสตรอว์เบอร์รี','คาเนชั่นรสจืด','จิตรลดาช็อกโกแลต'
 ,'จิตรลดารสจืด','จิตรลดารสหวาน','จิตรลดาสตรอว์เบอร์รี','ดัชมิลล์ไขมัน 0%','ดัชมิลล์ไขมันต่ำ','ดัชมิลล์ไฮโปรตีน','ดัชลล์อกกแลต'
 ,'ดัชมิลล์มอลต์','ดัชมิลล์รสกาแฟ','ดัชมิลล์รสจืด','ดัชมิลล์สตรอว์เบอร์รี','หนองโพไขมัน 0%','หนองโพไขมันต่ำ','หนองโพช็อกโกแลต'
 ,'หนองโพรสกาแฟ','หนองโพรสจืด','หนองโพรสหวาน','หนองโพสตรอว์เบอร์รี']

reduceCol = attrributes + stores 
reduceCol_all = attrributes + stores + milk_kind + products

In [ ]:
def reduced_keyword(wtkn,redCol):
    del_list = []
    ret_wtkn = wtkn
    for each in ret_wtkn:
        if each not in redCol:
            del_list.append(each)
    ret_wtkn = [x for x in ret_wtkn if x not in del_list]
    return ret_wtkn

In [ ]:
def freq_brand(x):
    #e = eval(x)
    e = x
    e.sort()
    f = [(k,len(list(g))) for k, g in groupby(e)]
    return f

In [ ]:
def concat_flavor(x,y):
    listflav = []
    for i in x:
        for j in y:
            listflav.append(i+j)
    return listflav

### 5.สร้าง (Reduce) Bag of word ด้วย dictionary.doc2bow จัดลง dataframe

In [ ]:
cursor_thread = col_thread.find()
df_thr_process = pd.DataFrame(cursor_thread)

In [ ]:
cursor_comment = col_comment.find()
df_cmt_process = pd.DataFrame(cursor_comment)

In [ ]:
# คัด headline ที่พูดถึง domain นม
df_urls_milk = df_thr_process[(df_thr_process.t_mention_dairy.str.len() != 0) | (df_thr_process.URLs.isin(execption_urls))]

In [ ]:
# เมื่อ join กันแล้ว คอมเม้นทุกคอมเม้นจะอยู่ใน domain นมทั้งหมด
df_join_url = pd.merge(df_urls_milk,df_cmt_process,how='inner',on='URLs')

In [ ]:
df_join_url.drop(columns=['_id_x','_id_y','headline_y'],inplace=True)
df_join_url.rename(columns={"headline_x":"headline"},inplace=True)

In [ ]:
# ทดลอง QAP mathod case2,4
df_join_url['token_text_reduce'] = df_join_url['token_text'].apply(lambda x: reduced_keyword(x, reduceCol))

# ทดลอง QAP mathod  case1,5
#df_join_url['token_text_reduce'] = df_join_url['token_text'].apply(lambda x: reduced_keyword(x, reduceCol_all))

In [ ]:
#df_join_url.set_index('commentId',inplace=True)
df_join_url.head(2)

In [ ]:
dictionary = gensim.corpora.Dictionary(df_join_url['token_text_reduce'])

In [ ]:
df_join_url['reduce_bow'] = df_join_url["token_text_reduce"].map(dictionary.doc2bow)
df_join_url['reduce_bow_txt'] = df_join_url["reduce_bow"].apply(lambda x:[(dictionary[id_], frequence) for id_, frequence in x])

In [ ]:
df_join_url.drop(columns=['Retrived_date','reduce_bow'],inplace=True)

In [ ]:
df_join_url.insert(3,'mention_domain',value=None)
df_join_url.insert(4,'mention_product',value=None)
df_join_url.insert(5,'mention_brand',value=None)

In [ ]:
df_join_url.tail()

Impute brand & product เข้า comment และสรุปก่อนเข้า BoW

In [ ]:
for i,e in enumerate(df_join_url.index,0):
    if (len(df_join_url.t_mention_dairy.iloc[i]) > 0) & (len(df_join_url.cmt_mention_dairy.iloc[i]) > 0):         # 1.
        df_join_url['mention_domain'].iloc[i] = df_join_url.cmt_mention_dairy.iloc[i]
    elif  (len(df_join_url.t_mention_dairy.iloc[i]) > 0) & (len(df_join_url.cmt_mention_dairy.iloc[i]) == 0):  # 2.
        df_join_url['mention_domain'].iloc[i] = df_join_url.t_mention_dairy.iloc[i]
    elif  (len(df_join_url.t_mention_dairy.iloc[i]) == 0) & (len(df_join_url.cmt_mention_dairy.iloc[i]) > 0):  # 3.
        df_join_url['mention_domain'].iloc[i] = df_join_url.cmt_mention_dairy.iloc[i]
    else:
        df_join_url['mention_domain'].iloc[i] = None

In [ ]:
for i,e in enumerate(df_join_url.index,0):
    if (len(df_join_url.t_mention_brand.iloc[i]) > 0) & (len(df_join_url.cmt_mention_brand.iloc[i]) > 0):         # 1.
        df_join_url['mention_brand'].iloc[i] = df_join_url.cmt_mention_brand.iloc[i]
    elif  (len(df_join_url.t_mention_brand.iloc[i]) > 0) & (len(df_join_url.cmt_mention_brand.iloc[i]) == 0):  # 2.
        df_join_url['mention_brand'].iloc[i] = df_join_url.t_mention_brand.iloc[i]
    elif  (len(df_join_url.t_mention_brand.iloc[i]) == 0) & (len(df_join_url.cmt_mention_brand.iloc[i]) > 0):  # 3.
        df_join_url['mention_brand'].iloc[i] = df_join_url.cmt_mention_brand.iloc[i]
    else:
        df_join_url['mention_brand'].iloc[i] = None

In [ ]:
for i,e in enumerate(df_join_url.index,0):
    if (len(df_join_url.t_mention_product.iloc[i]) > 0) & (len(df_join_url.cmt_mention_product.iloc[i]) > 0):         # 1.
        df_join_url['mention_product'].iloc[i] = df_join_url.cmt_mention_product.iloc[i]
    elif  (len(df_join_url.t_mention_product.iloc[i]) > 0) & (len(df_join_url.cmt_mention_product.iloc[i]) == 0):  # 2.
        df_join_url['mention_product'].iloc[i] = df_join_url.t_mention_product.iloc[i]
    elif  (len(df_join_url.t_mention_product.iloc[i]) == 0) & (len(df_join_url.cmt_mention_product.iloc[i]) > 0):  # 3.
        df_join_url['mention_product'].iloc[i] = df_join_url.cmt_mention_product.iloc[i]
    else:
        df_join_url['mention_product'].iloc[i] = None

In [ ]:
df_join_url.drop(columns=['t_mention_dairy','t_mention_product','t_mention_brand'
                          ,'cmt_mention_dairy','cmt_mention_product','cmt_mention_brand'],inplace=True)

concat Brand + Product มาต่อกันเพื่อระบุตราและยี่ห้อ เพื่อลงระดับ flavor

Create cooccurence matrix

In [ ]:
# comment ออกเมื่อไม่ concat brand+flavor
df_final = df_join_url.copy()
df_final.set_index('commentId',inplace=True)
df_final['mention_product'] = df_final['mention_product'].apply(lambda x: x if x is None else freq_brand(x))
df_final['mention_brand'] = df_final['mention_brand'].apply(lambda x: x if x is None else freq_brand(x))
df_final['mention_domain'] = df_final['mention_domain'].apply(lambda x: x if x is None else freq_brand(x))

In [ ]:
#df_brand_cooc = pd.concat([pd.DataFrame(s,columns=['w','c']).set_index('w') for s in df_final["mention_brand"]], axis=1, sort=False).fillna(0).T.set_index(df_final.index)
#df_prd_cooc = pd.concat([pd.DataFrame(s,columns=['w','c']).set_index('w') for s in df_final["mention_product"]], axis=1, sort=False).fillna(0).T.set_index(df_final.index)
df_cooc = pd.concat([pd.DataFrame(s,columns=['w','c']).set_index('w') for s in df_final["reduce_bow_txt"]], axis=1, sort=False).fillna(0).T.set_index(df_final.index)

In [ ]:
#df_brand_cooc.reset_index(inplace=True)
#df_prd_cooc.reset_index(inplace=True)
df_cooc.reset_index(inplace=True)

In [ ]:
# ทดลอง  QAP mathod case2,4: Brand vs Product vs Attribute (ไม่เอาแล้ว)
#join_df = pd.merge(df_brand_cooc, df_prd_cooc, on='commentId', how='inner', suffixes=(False, False))
#join_df = pd.merge(join_df, df_cooc, on='commentId', how='inner')

# ทดลอง  QAP mathod case3: Product vs Attribute (ไม่เอาแล้ว)
#join_df = pd.merge(df_prd_cooc, df_cooc, on='commentId', how='inner', suffixes=(False, False))

# ทดลอง  QAP mathod case1,5: Brand vs Attribute กรณีแยก ยี่ห้อ และ ผลิตภัณฑ์ จาก reduce_bow_txt
#join_df = pd.merge(df_brand_cooc, df_cooc, on='commentId', how='inner', suffixes=(False, False))

# ไม่ Join ทำในกรณีไม่แยก ยี่ห้อ และ ผลิตภัณฑ์ ออกจาก reduce_bow_txt
join_df = df_cooc.copy()

join_df.set_index('commentId',inplace=True)

In [ ]:
join_df.head()

Export ข้อมูล เพื่อนำไปสุ่มแถวทำ QAP

In [ ]:
join_df.to_csv(root_output_path+"/cooc_bow_all.csv",index=True)

## Source for graph generate

### 1.Create frequency co-occurrence matrix จาก Bag of word<br>
* ใช้สำหรับทำ MDS และโยนเข้า R (ทำ normalized บน R) ในลักษณะ Term-Term Cross tabulation<br> 
(คนละอย่างกับ TF-IDF cooc matrix ที่เป็น Word-word ซึ่งต้องรัน bigram เป็นราย document)

In [ ]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [ ]:
def cnt_Grpby(df,i,j):
    a = pd.DataFrame(df.iloc[:,i])
    b = pd.DataFrame(df.iloc[:,j])
    slice_df = pd.concat([a,b], axis=1, join="inner")
    slice_df.columns = ['A','B']
    cooc = slice_df[(slice_df.A>0)&(slice_df.B>0)]
    return cooc.shape[0]

In [ ]:
# Co-occurrence
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        if i != j:
            item_item_matrix.iloc[i,j] = cnt_Grpby(join_df,i,j)
        else:
            item_item_matrix.iloc[i,j] = 0

In [ ]:
item_item_matrix.head()

In [ ]:
item_item_matrix.to_excel(comment_cooc_freq_file, index=True)

### 2.Create co-occurrence matrix with Jaccard Distance สำหรับสร้างกราฟ

In [ ]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [ ]:
def jaccard_index(df,i,j):
    a = pd.DataFrame(df.iloc[:,i])
    b = pd.DataFrame(df.iloc[:,j])
    slice_df = pd.concat([a,b], axis=1, join="inner")
    slice_df.columns = ['A','B']
    a_in_b = slice_df[(slice_df.A>0)&(slice_df.B>0)]
    a_occur = slice_df[(slice_df.A>0)]
    b_occur = slice_df[(slice_df.B>0)]
    jacc_index = a_in_b.shape[0]/(a_occur.shape[0]+b_occur.shape[0]-a_in_b.shape[0])
    return jacc_index

In [ ]:
# Co-occurrence
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        item_item_matrix.iloc[i,j] = jaccard_index(join_df,i,j)
        #item_item_matrix.iloc[i,j] = jaccard_score(join_df.iloc[i],join_df.iloc[j], average='weighted')
#item_item_matrix.fillna(0,inplace=True)

In [ ]:
item_item_matrix.head()

In [ ]:
item_item_matrix.to_excel(comment_cooc_jacc_file, index=True)

In [ ]:
del item_item_matrix,join_df

## Source for Robustness Check & Cross Validation (7x7 brands matrix)

#### คำนวณ Index ของยี่ห้อต่อยี่ห้อ จากทุกๆ Attributes แสดงผลเป็น Matrix ขนาด 7x7

In [8]:
matrix_freq = pd.read_csv(root_output_path+"/cooc_bow_all.csv",index_col=0)
#cooc_matrix_freq = pd.read_excel(comment_cooc_freq_file,index_col=0)

In [9]:
matrix_freq.head()

,7-Eleven,tops,ขายไม่ดี,นม,รสหวาน,เมจิ,เบาหวาน,ซื้อ,กาแฟ,จืด,...,แมกนีเซียม,โรคท้องร่วง,สยามพารากอน,ห้างร้าน,โรคขาดอาหาร,ช่องแช่แข็ง,โยเกิร์ท,พารากอน,พาราไดซ์,โด๊บ
commentId,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
brands = ['เมจิ', 'ไทยเดนมาร์ค', 'แดรี่โฮม', 'โฟร์โมสต์', 'โชคชัย', 'หนองโพ', 'เอ็มมิลค์']

In [11]:
matrix_freq_brand = matrix_freq[brands]
matrix_freq_unbrand = matrix_freq.drop(labels=brands,axis=1,inplace=False)

In [ ]:
# Transpose frequency into Attribute Rating Format for precompute MDS
#cooc_matrix_freq = cooc_matrix_freq[brands]
#cooc_matrix_freq.drop(brands, axis=0, inplace=True)

In [ ]:
# จำนวน Attributes = 330, brands = 7
#cooc_matrix_freq.shape

### 1.Jaccard Index

In [12]:
# REF: Lee Soojung,(2017). Improving Jaccard Index for Measuring Similarity in Collaborative Filtering
# Information Science and Applications 2017, 124.
def jaccard_index_aggregate(df_b, df_attr , i, j, k):
    brand1 = pd.DataFrame(df_b.iloc[:,i])
    brand2 = pd.DataFrame(df_b.iloc[:,j])
    attr = pd.DataFrame(df_attr.iloc[:,k])
    slice_df = pd.concat([brand1, attr, brand2], axis=1, join="inner")
    a_in_b = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)&(slice_df.iloc[:,2]>0)]
    a_occur = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)]
    b_occur = slice_df[(slice_df.iloc[:,2]>0)&(slice_df.iloc[:,1]>0)]
    jacc_index = a_in_b.shape[0]/(a_occur.shape[0]+b_occur.shape[0]-a_in_b.shape[0])
    return jacc_index

In [13]:
# item_item_matrix = pd.DataFrame(index=cooc_matrix_freq.columns,columns=cooc_matrix_freq.columns).fillna(0)
item_item_matrix = pd.DataFrame(index=matrix_freq_brand.columns,columns=matrix_freq_brand.columns).fillna(0)

In [14]:
jacc = pd.DataFrame(columns=matrix_freq_unbrand.columns)
jacc = jacc.append(pd.Series(), ignore_index=True).fillna(0)

In [15]:
for i in range(0,len(matrix_freq_brand.columns)): # 0 to 6
    for j in range(0,len(matrix_freq_brand.columns)): # 0 to 6
        for k in range(0,len(matrix_freq_unbrand.columns)): # 0 to 329
            try:
                jacc.iloc[0,k] = jaccard_index_aggregate(matrix_freq_brand, matrix_freq_unbrand,i, j, k)
            except Exception:
                continue
        item_item_matrix.iloc[i,j] = jacc.mean(axis=1)[0]

In [16]:
#for i in range(0,len(cooc_matrix_freq.columns)):
#    for j in range(0,len(cooc_matrix_freq.columns)):
#        item_item_matrix.iloc[i,j] = jaccard_index(matrix_freq_brand,i,j)

In [17]:
item_item_matrix

,เมจิ,ไทยเดนมาร์ค,แดรี่โฮม,โฟร์โมสต์,โชคชัย,หนองโพ,เอ็มมิลค์
เมจิ,0.681818,0.038565,0.013546,0.071315,0.020269,0.024907,0.027511
ไทยเดนมาร์ค,0.038565,0.593939,0.025844,0.082011,0.015587,0.112704,0.000029
แดรี่โฮม,0.013546,0.025844,0.315152,0.000000,0.005926,0.016120,0.001389
โฟร์โมสต์,0.071315,0.082011,0.000000,0.584848,0.021927,0.093050,0.000159
โชคชัย,0.020269,0.015587,0.005926,0.021927,0.339394,0.009851,0.000000
หนองโพ,0.024907,0.112704,0.016120,0.093050,0.009851,0.430303,0.000000
เอ็มมิลค์,0.027511,0.000029,0.001389,0.000159,0.000000,0.000000,0.163636


In [18]:
item_item_matrix.to_excel(comment_robust_jaccard, index=True)

### 2.Lift normalization (Netzer et al., 2011)

In [19]:
def lift_index(df,i,j):
    a = pd.DataFrame(df.iloc[:,i])
    b = pd.DataFrame(df.iloc[:,j])
    slice_df = pd.concat([a,b], axis=1, join="inner")
    slice_df.columns = ['A','B']
    p_a_in_b = slice_df[(slice_df.A>0)&(slice_df.B>0)].shape[0]/slice_df.shape[0]
    p_a_occur = slice_df[(slice_df.A>0)].shape[0]/slice_df.shape[0]
    p_b_occur = slice_df[(slice_df.B>0)].shape[0]/slice_df.shape[0]
    p_b_comprem_occur = slice_df[(slice_df.B==0)].shape[0]/slice_df.shape[0]
    lift = p_a_in_b/(p_a_occur*p_b_comprem_occur)
    return lift

In [20]:
# Follow style of Lee Soojung,(2017)
def lift_index_aggregate(df_b, df_attr , i, j, k):
    brand1 = pd.DataFrame(df_b.iloc[:,i])
    brand2 = pd.DataFrame(df_b.iloc[:,j])
    attr = pd.DataFrame(df_attr.iloc[:,k])
    slice_df = pd.concat([brand1, attr, brand2], axis=1, join="inner")
    p_a_in_b = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)&(slice_df.iloc[:,2]>0)].shape[0]/slice_df.shape[0]
    p_a_occur = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)].shape[0]/slice_df.shape[0]
    p_b_occur = slice_df[(slice_df.iloc[:,2]>0)&(slice_df.iloc[:,1]>0)].shape[0]/slice_df.shape[0]
    p_b_comprem_occur = (slice_df.shape[0]-p_b_occur)/slice_df.shape[0]
    lift = p_a_in_b/(p_a_occur*p_b_comprem_occur)
    return lift

In [21]:
#item_item_matrix = pd.DataFrame(index=cooc_matrix_freq.columns,columns=cooc_matrix_freq.columns).fillna(0)
item_item_matrix = pd.DataFrame(index=matrix_freq_brand.columns,columns=matrix_freq_brand.columns).fillna(0)

In [22]:
lift = pd.DataFrame(columns=matrix_freq_unbrand.columns)
lift = lift.append(pd.Series(), ignore_index=True).fillna(0)

In [23]:
for i in range(0,len(matrix_freq_brand.columns)): # 0 to 6
    for j in range(0,len(matrix_freq_brand.columns)): # 0 to 6
        for k in range(0,len(matrix_freq_unbrand.columns)): # 0 to 329
            try:
                lift.iloc[0,k] = lift_index_aggregate(matrix_freq_brand, matrix_freq_unbrand,i, j, k)
            except Exception:
                continue
        item_item_matrix.iloc[i,j] = lift.mean(axis=1)[0]

In [24]:
#for i in range(0,len(item_item_matrix.columns)):
#    for j in range(0,len(item_item_matrix.columns)):
#        item_item_matrix.iloc[i,j] = lift_index(cooc_matrix_freq,i,j)

In [25]:
item_item_matrix

,เมจิ,ไทยเดนมาร์ค,แดรี่โฮม,โฟร์โมสต์,โชคชัย,หนองโพ,เอ็มมิลค์
เมจิ,0.681818,0.056653,0.014080,0.087099,0.024433,0.028420,0.027527
ไทยเดนมาร์ค,0.109314,0.600320,0.040171,0.118685,0.026533,0.137341,0.006410
แดรี่โฮม,0.068916,0.086756,0.321532,0.006380,0.019123,0.046581,0.007861
โฟร์โมสต์,0.183612,0.127704,0.004721,0.589570,0.032548,0.118287,0.004881
โชคชัย,0.094566,0.056734,0.008488,0.046703,0.340933,0.028807,0.001539
หนองโพ,0.112174,0.226799,0.022079,0.173191,0.015940,0.431842,0.001539
เอ็มมิลค์,0.147349,0.003030,0.004419,0.003030,0.000000,0.000000,0.163636


In [26]:
item_item_matrix.to_excel(comment_robust_lift, index=True)

### 3. Inclusion Index (Qin He, 1999)

In [27]:
def inclu_index(df,i,j):
    a = pd.DataFrame(df.iloc[:,i])
    b = pd.DataFrame(df.iloc[:,j])
    slice_df = pd.concat([a,b], axis=1, join="inner")
    slice_df.columns = ['A','B']
    a_in_b = slice_df[(slice_df.A>0)&(slice_df.B>0)].shape[0]
    a_occur = slice_df[(slice_df.A>0)].shape[0]
    b_occur = slice_df[(slice_df.B>0)].shape[0]
    inclu = a_in_b/min(a_occur,b_occur)
    return inclu

In [28]:
def inclu_index_aggregate(df_b, df_attr , i, j, k):
    brand1 = pd.DataFrame(df_b.iloc[:,i])
    brand2 = pd.DataFrame(df_b.iloc[:,j])
    attr = pd.DataFrame(df_attr.iloc[:,k])
    slice_df = pd.concat([brand1, attr, brand2], axis=1, join="inner")
    a_in_b = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)&(slice_df.iloc[:,2]>0)].shape[0]
    a_occur = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)].shape[0]
    b_occur = slice_df[(slice_df.iloc[:,2]>0)&(slice_df.iloc[:,1]>0)].shape[0]
    inclu = a_in_b/min(a_occur,b_occur)
    return inclu

In [30]:
#item_item_matrix = pd.DataFrame(index=cooc_matrix_freq.columns,columns=cooc_matrix_freq.columns).fillna(0)
item_item_matrix = pd.DataFrame(index=matrix_freq_brand.columns,columns=matrix_freq_brand.columns).fillna(0)

In [31]:
inclu = pd.DataFrame(columns=matrix_freq_unbrand.columns)
inclu = inclu.append(pd.Series(), ignore_index=True).fillna(0)

In [32]:
for i in range(0,len(matrix_freq_brand.columns)): # 0 to 6
    for j in range(0,len(matrix_freq_brand.columns)): # 0 to 6
        for k in range(0,len(matrix_freq_unbrand.columns)): # 0 to 329
            try:
                inclu.iloc[0,k] = inclu_index_aggregate(matrix_freq_brand, matrix_freq_unbrand,i, j, k)
            except Exception:
                continue
        item_item_matrix.iloc[i,j] = inclu.mean(axis=1)[0]

In [42]:
#for i in range(0,len(item_item_matrix.columns)):
#    for j in range(0,len(item_item_matrix.columns)):
#        item_item_matrix.iloc[i,j] = incurr_index(cooc_matrix_freq,i,j)

In [33]:
item_item_matrix

,เมจิ,ไทยเดนมาร์ค,แดรี่โฮม,โฟร์โมสต์,โชคชัย,หนองโพ,เอ็มมิลค์
เมจิ,0.681818,0.294119,0.286575,0.303484,0.272889,0.271024,0.363523
ไทยเดนมาร์ค,0.254725,0.736364,0.519706,0.370596,0.350266,0.463958,0.401125
แดรี่โฮม,0.379750,0.396012,0.621760,0.364185,0.370866,0.388510,0.374513
โฟร์โมสต์,0.408029,0.377162,0.323175,0.781818,0.520402,0.525621,0.473481
โชคชัย,0.483201,0.451388,0.428146,0.438079,0.726768,0.486281,0.458085
หนองโพ,0.489711,0.603351,0.512904,0.552894,0.479210,0.784848,0.663636
เอ็มมิลค์,0.804924,0.678788,0.680177,0.672727,0.663636,0.663636,0.821212


In [34]:
item_item_matrix.to_excel(comment_robust_incur, index=True)

### 4.cosine similarity correlation matrix

In [51]:
def cosine_aggregate(df_b, df_attr , i, j, k):
    brand1 = pd.DataFrame(df_b.iloc[:,i])
    brand2 = pd.DataFrame(df_b.iloc[:,j])
    attr = pd.DataFrame(df_attr.iloc[:,k])
    slice_df = pd.concat([brand1, attr, brand2], axis=1, join="inner")
    a_occur = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)]
    b_occur = slice_df[(slice_df.iloc[:,2]>0)&(slice_df.iloc[:,1]>0)]
    cosin = 
    return cosin

In [52]:
cosin = pd.DataFrame(columns=matrix_freq_unbrand.columns)
cosin = cosin.append(pd.Series(), ignore_index=True).fillna(0)

In [53]:
#item_item_matrix = pd.DataFrame(index=cooc_matrix_freq.columns,columns=cooc_matrix_freq.columns).fillna(0)
item_item_matrix = pd.DataFrame(index=matrix_freq_brand.columns,columns=matrix_freq_brand.columns).fillna(0)

In [54]:
for i in range(0,len(matrix_freq_brand.columns)): # 0 to 6
    for j in range(0,len(matrix_freq_brand.columns)): # 0 to 6
        for k in range(0,len(matrix_freq_unbrand.columns)): # 0 to 329
            try:
                cosin.iloc[0,k] = cosine_aggregate(matrix_freq_brand, matrix_freq_unbrand,i, j, k)
            except Exception:
                continue
        item_item_matrix.iloc[i,j] = cosin.mean(axis=1)[0]

In [55]:
# Co-occurrence
#for i in range(0,len(item_item_matrix.columns)):
#    for j in range(0,len(item_item_matrix.columns)):
#        item_item_matrix.iloc[i,j] = cosine(cooc_matrix_freq.iloc[i],cooc_matrix_freq.iloc[j])
#item_item_matrix.fillna(0,inplace=True)

In [56]:
item_item_matrix

,เมจิ,ไทยเดนมาร์ค,แดรี่โฮม,โฟร์โมสต์,โชคชัย,หนองโพ,เอ็มมิลค์
เมจิ,0.000000,0.008411,0.015603,0.025820,0.034938,0.032437,0.032437
ไทยเดนมาร์ค,0.031482,0.023071,0.034353,0.053237,0.055263,0.056285,0.057800
แดรี่โฮม,0.055701,0.058850,0.040497,0.045600,0.056503,0.068716,0.070788
โฟร์โมสต์,0.072036,0.072906,0.072906,0.041608,0.059704,0.063969,0.066999
โชคชัย,0.070056,0.068070,0.068070,0.069352,0.029592,0.048222,0.051252
หนองโพ,0.049756,0.047190,0.048635,0.046983,0.051252,0.008133,0.009951
เอ็มมิลค์,0.010508,0.010106,0.010106,0.010508,0.009951,0.009951,0.000000


In [57]:
item_item_matrix.to_excel(comment_robust_cosine, index=True)

### 5.สุ่มตัดจำนวน Row-Col เพื่อนำไปทำ QAP Test (with Jaccard Index)

สุ่ม Document 1/16, 1/8, 1/4, 1/2 เอามาทำ Cooc-matrix แล้วไล่ทำ QAP เทียบ 9000 doc กับ 4 ชุด dataset

In [ ]:
np.seterr(all='ignore')
df_cooc_frq = pd.read_csv(root_output_path+"/cooc_bow_all.csv",index_col=0)

In [ ]:
df_rand1 = df_cooc_frq.sample(frac=(1/16))
df_rand2 = df_cooc_frq.sample(frac=(1/8))
df_rand3 = df_cooc_frq.sample(frac=(1/4))
df_rand4 = df_cooc_frq.sample(frac=(1/2))

In [ ]:
print(df_rand1.shape[0],df_rand2.shape[0],df_rand3.shape[0],df_rand4.shape[0])

In [ ]:
def create_RandomcoccMat_Lift(df):
    item_item_matrix = pd.DataFrame(index=df.columns,columns=df.columns).fillna(0)
    for i in range(0,len(item_item_matrix.columns)) :
        for j in range(0,len(item_item_matrix.columns)) :
            A = df.iloc[:,i]
            B = df.iloc[:,j]
            item_item_matrix.iloc[i,j] = lift_score(A,B)
    item_item_matrix.replace([np.inf, -np.inf], np.nan,inplace=True)
    item_item_matrix.fillna(0,inplace=True)
    return item_item_matrix

In [ ]:
item_item_matrix = create_RandomcoccMat_Lift(df_rand1)
item_item_matrix.to_excel('lift1.xlsx', index=True)

In [ ]:
item_item_matrix = create_RandomcoccMat_Lift(df_rand2)
item_item_matrix.to_excel('lift2.xlsx', index=True)

In [ ]:
item_item_matrix = create_RandomcoccMat_Lift(df_rand3)
item_item_matrix.to_excel('lift3.xlsx', index=True)

In [ ]:
item_item_matrix = create_RandomcoccMat_Lift(df_rand4)
item_item_matrix.to_excel('lift4.xlsx', index=True)